# Extended figure 6f part 1

In [ ]:
import sys
import subprocess

#import pkg_resources
#required = {'harmonypy','sklearn','scanpy','pandas', 'numpy', 'bbknn', 'scipy', 'matplotlib', 'seaborn' ,'scipy'}
#installed = {pkg.key for pkg in pkg_resources.working_set}
#missing = required - installed
#if missing:
#    print("Installing missing packages:" )
#    print(missing)
#    python = sys.executable
#    subprocess.check_call([python, '-m', 'pip', 'install', *missing], stdout=subprocess.DEVNULL)

%matplotlib inline
from collections import Counter
from collections import defaultdict
import scanpy as sc
import pandas as pd
import pickle as pkl
import numpy as np
from bbknn import bbknn
import scipy
import matplotlib.pyplot as plt
import re
import glob
import os
import sys
from geosketch import gs
from numpy import cov
import scipy.cluster.hierarchy as spc
import seaborn as sns; sns.set(color_codes=True)
from sklearn.linear_model import LogisticRegression
import sklearn
import harmonypy as hm
from pathlib import Path

sc.settings.verbosity = 3  # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.settings.set_figure_params(dpi=80, color_map='viridis')

# LR script

# Only block to edit in LR script

In [ ]:
# Introduce variables
# Note that this script expects raw data to be in "non-batch-corrected" adata.raw.X. 

# Required: Introduce the path you'd like to save figures or data to 
save_path = "/home/jovyan/mount_farm/nfs/team298/ar32/YS/brain_objects/kriegstein/Cellbender_from_Jimmy/cellbender_from_jimmy/training_on_all_brain_cellbender_with_model_already_made/LR_indiv/LR_indiv_outs/"

# Required: Name of second object
data1 = "_training"
data1_identifier = "YS"
# Provide path to obj2 // prediction/projection data
Object1 = "/home/jovyan/YS_project/YS/Data_objects/final_objects/A4_V7_YS_integrated_data_singlets_with_raw_counts_for_MS_plotting_20211111_with_obsp.h5ad"
# Provide categorical to join between datasets
#cat1 = "celltypes_for_comparison"

# Required: Name of second object
data2 = "brain_cellbender"
# Provide path to obj2 // prediction/projection data
Object2 = "/nfs/team298/ar32/YS/brain_objects/kriegstein/Cellbender_from_Jimmy/processed_object_20220329/raw_basic_batch_brain_cellbender_20220329.h5ad"
# Provide categorical to join between datasets
#cat2 = "leiden_res1"


# Required: LR Model Options
penalty='elasticnet' # can be ["l1","l2","elasticnet"]
sparcity=0.2
max_iter = 1000 #Increase if experiencing max iter issues
l1_ratio = 0.5 #If using elasticnet, tis controls the ratio between l1 and l2

# Optional: Batch correction options (this is for correction of eventual combined dataset for data1 and data2)
# If you do not have a batch variable for either data1 or data2, please add a "filler" column in the relevent adata.obs
# for the purposes of batch_correction and batch args below.
# e.g., adata.obs["whatever"] = "something"; batch="whatever"
batch_correction = "Harmony" # Will accept Harmony, BBKNN or False as options
#batch = ["fetal.ids", "orig.ident"] # Will accept any batch categorical. Comma space a batch categorical for each dataset. Position 1 is for data1, position 2 is for data2

# Optional: miscellaneous Options.   
subsample_train = True # Samples the training data to the smallest fraction (highly dependent on resolution of input celltype categorical). This corrects for proportional differences between celltype labels of interest in the training data. E.g., training data has 50,000 B cells, 20,000 T cells and 100 HSCs. This function will subsample all training to 100 cells per cell type. 
subsample_prop = 0.2 # Give this option a proprtion to subsample to(e.g 0.2), if NA given, will subsample to smallest population
subsample_predict = False
subsample_prop_predict = 0.5
remove_non_high_var = False

train_x = 'X' # Define the resource to train and predict on, PCA, X or UMAP (#if you wish to use gene expression, train_x = 'X')
remove_effect_of_custom_gene_list = '' # "./cell_cycle_genes.csv" #remove a custom list of genes from just the variable genes to compute PCA from. Your .csv should have HGNC gene names in the first column to be read in as a vector, any column name is fine.
use_raw = True # Do you want to use adata.raw.X (recommended)

# Rest of LR script

## Combining data and Preprocess

In [ ]:
# Check if filepaths are good
if not os.path.exists(save_path):
    os.makedirs(save_path)
    
if (Path(Object1).is_file() & Path(Object2).is_file()):
    print("adata file paths detetcted, proceeding to load")
    adata = sc.read(Object1)
    adata2 = sc.read(Object2)
    del adata.uns
    del adata2.uns
else: 
    raise TypeError("one or more .h5ad paths cannot be accessed")

# altering scanpy setting so that we can save it to our defined directory
sc._settings.ScanpyConfig(figdir=save_path)

# Combine and pre-process data to match correlations across PCA

# Module to detect shape mismatch and alternatively rebuild adata
if(use_raw==True):
    print('option detected to use raw data, proceeding to check if raw exists and if it matches data.X')
    if (hasattr(adata.raw, "X")):
        try: adata.X =  adata.raw.X  ; print('no mismatch in shape for adata detected')
        except: print("adata.X shape mismatched with adata.raw.X, proceeding to re-build data") ; adata = adata.raw.to_adata()
    else:
        print("no raw data detected in adata! proceeding to create raw partition from adata.X")
        adata.raw = adata
        
    if (hasattr(adata2.raw, "X")):
        try: adata2.X = adata2.raw.X ; print('no mismatch in shape for adata2 detected')
        except: print("adata2.X shape mismatched with adata.raw.X, proceeding to re-build data") ; adata2 = adata2.raw.to_adata()
    else:
        print("no raw data detected in adata! proceeding to create raw partition from adata.X")
        adata2.raw = adata2
        
    
# Define intersecting genes between datasets
adata_genes = list(adata.var.index)
adata2_genes = list(adata2.var.index)
keep_SC_genes = list(set(adata_genes) & set(adata2_genes))
print("keep gene list = " , len(keep_SC_genes), "adata1 gene length = ", len(adata_genes) , "adata2 gene length = ", len(adata2_genes) )

# Remove non-intersecting genes (this step will remove cite-seq data if training data is pure RNA seq)
adata_intersect1 = adata[:, keep_SC_genes]
adata = adata_intersect1
adata_intersect2 = adata2[:, keep_SC_genes]
adata2 = adata_intersect2

In [ ]:
del adata.obsm
del adata2.obsm

In [ ]:
adata

In [ ]:
adata2

In [ ]:
adata.obs['Dataset'] = adata.obs['fetal.ids']
adata2.obs['Dataset'] = adata2.obs['Object_orig']

In [ ]:
batch = ["Dataset", "Dataset"]

In [ ]:
adata.obs

In [ ]:
adata2.var[adata2.var_names.str.contains('P2RY')]

# gene of interest is in data!!

In [ ]:
adata.obs['cell.labels'].value_counts()

In [ ]:
cat1 = 'cell.labels'   # since only care about macrophage and microglia using broad 
cat2 = 'leiden_res1.5'

In [ ]:
%%time


# Optional subsampling of training data to 
if(subsample_train == True):
    
    if not(subsample_prop=="NA"):
        print("option to subsample by proportion chosen")
        prop = subsample_prop
        data = adata.obs[:]
        grouped = data.groupby(cat1)
        df = grouped.apply(lambda x: x.sample(frac=prop))
        df = df.droplevel(cat1)
        keep = df.index
        adata = adata[adata.obs.index.isin(keep)]
    else:
        print("subsample by smallest population")
        data = adata.obs
        data = data.sample(frac=1).groupby(cat1).head(min(adata.obs.groupby(cat1).size()))
        keep = data.index
        adata = adata[adata.obs.index.isin(keep)]
        
# Optional subsampling of training data to 
if(subsample_predict == True):
    if not(subsample_prop_predict=="NA"):
        print("option to subsample by proportion chosen")
        prop = subsample_prop_predict
        data = adata2.obs[:]
        grouped = data.groupby(cat2)
        df = grouped.apply(lambda x: x.sample(frac=prop))
        df = df.droplevel(cat2)
        keep = df.index
        adata2 = adata2[adata2.obs.index.isin(keep)]
    else:
        print("subsample by smallest population")
        data = adata2.obs
        data = data.sample(frac=1).groupby(cat2).head(min(adata.obs.groupby(cat2).size()))
        keep = data.index
        adata2 = adata2[adata2.obs.index.isin(keep)]

# Create a common batch column and do simple sanity check for batch variables
if not((batch_correction == "False") and (len(batch)>1)):
    print("Batch correction option detected, proceeding to format batch variables")
    batch_var = "lr_batch"
    adata.obs["lr_batch"] = adata.obs[batch[0]]
    adata2.obs["lr_batch"] = adata2.obs[batch[1]]
else: raise TypeError("Batch correction option detected but requires at least one categorical for each dataset!")

# Create a common obs column in both datasets containing the data origin tag
common_cat = "corr_concat" 
adata.obs[common_cat] = adata.obs[cat1].astype(str) + data1
adata2.obs[common_cat] = adata2.obs[cat2].astype(str) + data2
adata.obs = adata.obs.astype('category')
adata2.obs = adata2.obs.astype('category')

sc.pp.normalize_per_cell(adata, counts_per_cell_after=1e4)
sc.pp.log1p(adata)

sc.pp.log1p(adata2)

concat = adata2.concatenate(adata, join='inner',index_unique=None, batch_categories=None)
adata = concat[:]
#sc.pp.normalize_per_cell(adata, counts_per_cell_after=1e4)
#sc.pp.log1p(adata)
sc.pp.highly_variable_genes(adata, min_mean=0.1, max_mean=4)
#sc.pp.scale(adata, zero_center=False, max_value=None, copy=False) #zero_center=True (densifies output)

# Optionally remove genes of known confounding effect from variable list
if not (Path(remove_effect_of_custom_gene_list).is_file()):
    print("Custom gene list option is not selected or path is not readbale, proceeding with no variable removal")
else: 
    print("Custom gene removal list detected, proceeding to remove intersect from variable genes")
    regress_list = pd.read_csv(remove_effect_of_custom_gene_list)
    regress_list = regress_list.iloc[:, 0]
    adata.var["highly_variable"][adata.var.index.isin(regress_list)] = "False"

#Optionally remove genes that do not contribute to variance in combined data::Use only if training and predicting withsim reduced data    
if(remove_non_high_var==True):
    high_var = list(adata.var["highly_variable"][adata.var["highly_variable"]==True])
    adata = adata[:, adata.var["highly_variable"].isin(high_var)]   

# Now compute PCA
sc.pp.pca(adata, n_comps=50, use_highly_variable=True, svd_solver='arpack')

# Batch correction options
# The script will test later which Harmony values we should use
if not(batch_correction == "False"):
    sc.pp.neighbors(adata, n_neighbors=15, n_pcs=50)    
if(batch_correction == "Harmony"):
    print("Commencing harmony")
    # Create hm subset
    adata_hm = adata[:]
    # Set harmony variables
    data_mat = np.array(adata_hm.obsm["X_pca"])
    meta_data = adata_hm.obs
    vars_use = [batch_var]
    # Run Harmony
    ho = hm.run_harmony(data_mat, meta_data, vars_use, theta=3)
    res = (pd.DataFrame(ho.Z_corr)).T
    res.columns = ['X{}'.format(i + 1) for i in range(res.shape[1])]
    # Insert coordinates back into object
    adata_hm.obsm["X_pca_back"]= adata_hm.obsm["X_pca"][:]
    adata_hm.obsm["X_pca"] = np.array(res)
    # Run neighbours
    sc.pp.neighbors(adata_hm, n_neighbors=15, n_pcs=50)
    adata = adata_hm[:]
    del adata_hm
elif(batch_correction == "BBKNN"):
    print("Commencing BBKNN")
    sc.external.pp.bbknn(adata, batch_key=batch_var, approx=True, metric='angular', copy=False, n_pcs=50, trim=None, n_trees=10, use_faiss=True, set_op_mix_ratio=1.0, local_connectivity=15) 
    
print("adata1 and adata2 are now combined and preprocessed in 'adata' obj - success!")

In [ ]:
adata_save = adata[:]

In [ ]:
sc.tl.umap(adata)

In [ ]:
sc.pl.umap(adata, color='lr_batch')

In [ ]:
    # Define the separator category in the column of interest, this works by partial matches and enables a-symmetric 
    # comparisons
    Data1_group = data1
    Data2_group = data2
    # Define the common .obs column between concatinated data
    common_cat = "corr_concat"

    # This block defines subset_predict and subset_train and also runs LR_compare function
    group1 = (adata.obs[common_cat][adata.obs[common_cat].str.contains(Data1_group)]).unique()
    group1 = list(group1)
    group2 = (adata.obs[common_cat][adata.obs[common_cat].str.contains(Data2_group)]).unique()
    group2 = list(group2)
    subset_predict = np.array(adata.obs[common_cat].isin(group2))
    subset_train = np.array(adata.obs[common_cat].isin(group1))
    train_label = (adata.obs[common_cat][adata.obs[common_cat].isin(group1)]).values
    penalty = "elasticnet"
    common_cat = 'corr_concat'
    col_name='predicted'
    train_x = 'X'
    # adata - training+prediction adata object (combined). Pre-processed already
    # sparsity - larger sparsity, more bins, more conservative predictions, less accurate. Low sparist for clean output
                # A value of 0.2 is reasonable for L2 ridge regression
    # penalty - acts as buffer for assigning bins too harshly
    # train_x - arg refers to where you would like to derive your training reference from, i.e., GEX (X) or/elif.
                # PCA/UMAP in obsm. The two 'if' statements below handle train_x differently based on this
                # Based on train_x, the loops below compute 'train_label' (cell type values in training/landscape data) 
                # and 'predict_x'(prediction data equivalent of train_x)
    # train_label - cell type values in training/landscape data
    # subset_predict - mandatory subset of predict_x which contains metadata for expression
    # subset_train - mandatory subset of train_x which contains metadata for expression
    
    # Redefine LR parameters 'penalty' and 'sparsity' if you would like to deviate from defaults set above
    
    # Assign 'lr' as sklearn logistic regression func, with penalty and sparsity defined above
    lr = LogisticRegression(penalty = penalty, C = sparcity, max_iter =  max_iter)
    
    if (penalty == "l1"):
        lr = LogisticRegression(penalty = penalty, C = sparcity, max_iter =  max_iter, dual = True, solver = 'liblinear',multi_class = 'ovr' ) # one-vs-rest
    if (penalty == "elasticnet"):
        lr = LogisticRegression(penalty = penalty, C = sparcity, max_iter =  max_iter, dual=False,solver = 'saga',l1_ratio=l1_ratio,multi_class = 'multinomial')

    if train_x == 'X':
        # Define training parameters
        train_label = adata.obs[common_cat].values
        predict_label = train_label[subset_predict]
        train_label = train_label[subset_train]
        train_x = adata.X[adata.obs.index.isin(list(adata.obs[subset_train].index))]
        predict_x = adata.X[adata.obs.index.isin(list(adata.obs[subset_predict].index))]

    elif train_x in adata.obsm.keys():
        # Define training parameters
        train_label = adata.obs[common_cat].values
        predict_label = train_label[subset_predict]
        train_label = train_label[subset_train]
        train_x = adata.obsm[train_x]
        predict_x = train_x
        train_x = train_x[subset_train, :]
        # Define prediction parameters
        predict_x = predict_x[subset_predict]
        predict_x = pd.DataFrame(predict_x)
        predict_x.index = adata.obs[subset_predict].index

    # Train predictive model using user defined partition labels (train_x ,train_label, predict_x)
    lr.fit(train_x, train_label)
    predict_proba = lr.predict_proba(predict_x)

    # Create prediction table and map to adata.obs (in adata.obs["predict"] in the combined object), for the cells that
    # are in predict dataset
    predict = lr.predict(predict_x)
    predict = pd.DataFrame(predict)
    predict.index = adata.obs[subset_predict].index
    col_name='predicted'
    adata.obs[col_name] = adata.obs.index
    adata.obs[col_name] = adata.obs[col_name].map(predict[0])

In [ ]:
# Function to plot heatmap by percentage
def plot_df_heatmap(df, cmap='viridis', title=None, figsize=(7, 7), rotation=90, save=None, **kwargs):
    fig, ax = plt.subplots(figsize=figsize)
    im = ax.imshow(df, cmap=cmap, aspect='auto', **kwargs)
    if 0 < rotation < 90:
        horizontalalignment = 'right'
    else:
        horizontalalignment = 'center'
    plt.xticks(
        range(len(df.columns)),
        df.columns,
        rotation=rotation,
        horizontalalignment=horizontalalignment,
    )
    plt.yticks(range(len(df.index)), df.index)
    if title:
        fig.suptitle(title)
    #fig.colorbar(im)
    if save:
        plt.savefig(fname=save, bbox_inches='tight', pad_inches=0.1)

# Plot probability table by html
def cross_table(adata, x, y, normalise=None, highlight=False, subset=None):                                                                                                                                                                                              
    """Make a cross table comparing two categorical annotations
    """
    x_attr = adata.obs[x]
    y_attr = adata.obs[y]
    if subset is not None:
        x_attr = x_attr[subset]
        y_attr = y_attr[subset]
    crs_tbl = pd.crosstab(x_attr, y_attr)
    if normalise == 'x':
        x_sizes = x_attr.groupby(x_attr).size().values
        crs_tbl = (crs_tbl.T / x_sizes).round(2).T
    elif normalise == 'y':
        y_sizes = x_attr.groupby(y_attr).size().values
        crs_tbl = (crs_tbl / y_sizes).round(2)
    if highlight:
        return crs_tbl.style.background_gradient(cmap='viridis', axis=0)
    return crs_tbl

In [ ]:
import pickle

In [ ]:
filename = '/home/jovyan/mount_farm/nfs/team298/ar32/YS/brain_objects/kriegstein/Cellbender_from_Jimmy/cellbender_from_jimmy/training_on_all_brain_cellbender_with_model_already_made/LR_indiv/LR_indiv_outs/microglia_LR_model_20220404.sav'
pickle.dump(lr, open(filename, 'wb'))

In [ ]:
model = lr

In [ ]:
for i in adata_save.obs.columns:
    adata_save.obs[i] = adata_save.obs[i].astype('category')

In [ ]:
adata_save.write('combined_YS_brain_cellbender_LR_20220404.h5ad')

In [ ]:
run_date='20220404'

In [ ]:
train_label = adata.obs[common_cat].values
predict_label = train_label[subset_predict]

pred_out = pd.DataFrame(model.predict(predict_x),columns = ['predicted'],index = adata.obs.index[adata.obs[common_cat].isin(group2)])
pred_out['orig_labels'] = predict_label
proba = pd.DataFrame(model.predict_proba(predict_x),columns = lr.classes_,index = adata.obs.index[adata.obs[common_cat].isin(group2)])
pred_out = pred_out.join(proba)

pred_out.to_csv(save_path + '/pred_out_' + run_date + '.csv')



model_mean_probs = pred_out.loc[:, pred_out.columns != 'predicted'].groupby('orig_labels').mean()
#model_mean_probs = pred_out.loc[:, pred_out.columns != 'predicted'].groupby('orig_labels').median()
model_mean_probs = model_mean_probs #*100
model_mean_probs = model_mean_probs.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
crs_tbl = model_mean_probs.copy()
# Sort df columns by rows
crs_tbl = crs_tbl.sort_values(by =list(crs_tbl.index), axis=1,ascending=False)



# Plot_df_heatmap(crs_tbl, cmap='coolwarm', rotation=90, vmin=20, vmax=70)
pal = sns.diverging_palette(240, 10, n=10)
plt.figure(figsize=(20,15))

sns.set(font_scale=0.8)
g = sns.heatmap(crs_tbl, cmap=pal,  annot=True,vmin=0, vmax=1, linewidths=1, center=0.5, square=True, cbar_kws={"shrink": 0.5})
plt.ylabel("Original labels")
plt.xlabel("Training labels")

# pull out microglia and macrophage

In [ ]:
df = adata.obs[adata.obs.index.isin(list(pred_out.index))]
df

In [ ]:
df.columns

In [ ]:
pred_out_backup = pred_out.copy()
adata_save_backup = adata_save.copy()

In [ ]:
sc.pl.umap(adata, color='lr_batch')

In [ ]:
pred_out['predicted'].value_counts()

In [ ]:
genes_to_plot = [
'CD4',
'CD14',
'C1QA',
'TREM2',
'CX3CR1',
'P2RY12'
]

In [ ]:
sc.pl.dotplot(adata, groupby='predicted', var_names=genes_to_plot, swap_axes = True)

# transfer clusters over made on broad 

In [ ]:
metadata = pd.read_csv('/nfs/team298/ar32/YS/brain_objects/kriegstein/Cellbender_from_Jimmy/cellbender_from_jimmy/training_on_all_brain_cellbender_with_model_already_made/adata_metadata_with_microglia_clus_predicted_20220404.csv', index_col=0)

In [ ]:
metadata

In [ ]:
adata_backup = adata[:]

In [ ]:
# Plotting purposes - Subset data to only contain prediction data with new predicted labelså

# Make adata the predicted data with projected annotations in adata.obs["predicted"]. "corr_concat" contains old 
# cell.labels and dataset name combined 
common_cat = "corr_concat"
adata_concat = adata[:]
adata = adata[adata.obs["predicted"].isin(group1)]
adata = adata[adata.obs["corr_concat"].isin(group2)]

In [ ]:
adata

In [ ]:
list(adata.obs.index[adata.obs.index.isin(adata.obs.index)]) == list(metadata.index)

In [ ]:
adata.obs['Microglia_check_res_20_clus_prediction'] = metadata['Microglia_check_res_20_clus_prediction']

In [ ]:
adata.obs['Microglia_check_res_20_clus_prediction'].value_counts()

In [ ]:
x = 'predicted'
y =  "Microglia_check_res_20_clus_prediction"

y_attr = adata.obs[y]
x_attr = adata.obs[x]
crs = pd.crosstab(x_attr, y_attr)
crs_tbl = crs
for col in crs_tbl :
    crs_tbl[col] = crs_tbl[col].div(crs_tbl[col].sum(axis=0)).multiply(100).round(2)
    
# Sort df columns by rows
crs_tbl = crs_tbl.sort_values(by =list(crs_tbl.index), axis=1,ascending=False)

# Plot_df_heatmap(crs_tbl, cmap='coolwarm', rotation=90, vmin=20, vmax=70)
pal = sns.diverging_palette(240, 10, n=10)
plt.figure(figsize=(20,15))
sns.set(font_scale=0.8)
g = sns.heatmap(crs_tbl[['Macrophage_clus_prediction','Microglia_clus_prediction']], cmap=pal, vmin=0, vmax=100, linewidths=1, center=50, square=True, cbar_kws={"shrink": 0.5}, annot=False)
plt.xlabel("Original labels")
plt.ylabel("Predicted labels")
#plt.savefig(save_path + "/LR_predictions_myeloid_clus_predict_labels.pdf")
#crs_tbl.to_csv(save_path + "/crs_tbl_myeloid_clus_predict_labels.csv")

In [ ]:
x = 'predicted'
y =  "predicted"

y_attr = adata.obs[y]
x_attr = adata.obs[x]
crs = pd.crosstab(x_attr, y_attr)
crs_tbl = crs
for col in crs_tbl :
    crs_tbl[col] = crs_tbl[col].div(crs_tbl[col].sum(axis=0)).multiply(100).round(2)
    
# Sort df columns by rows
crs_tbl = crs_tbl.sort_values(by =list(crs_tbl.index), axis=1,ascending=False)

# Plot_df_heatmap(crs_tbl, cmap='coolwarm', rotation=90, vmin=20, vmax=70)
pal = sns.diverging_palette(240, 10, n=10)
plt.figure(figsize=(20,15))
sns.set(font_scale=0.8)
g = sns.heatmap(crs_tbl[['Macrophage_training','Microglia_training']], cmap=pal, vmin=0, vmax=100, linewidths=1, center=50, square=True, cbar_kws={"shrink": 0.5}, annot=False)
plt.xlabel("Original labels")
plt.ylabel("Predicted labels")
#plt.savefig(save_path + "/LR_predictions_myeloid_clus_predict_labels.pdf")
#crs_tbl.to_csv(save_path + "/crs_tbl_myeloid_clus_predict_labels.csv")

# refind microglia in new embedding

In [ ]:
adata_save

In [ ]:
adata

In [ ]:
adata = adata_save[:]

In [ ]:
adata

In [ ]:
res = 5
key_add = 'leiden_res5'
adata.obs[key_add] = "nan"
sc.tl.leiden(adata, resolution= res, key_added= key_add, random_state=26, n_iterations=-1)

In [ ]:
adata.obs

In [ ]:
pred_out

In [ ]:
adata = adata[adata.obs.index.isin(list(pred_out.index))]

In [ ]:
adata

In [ ]:
list(adata.obs.index[adata.obs.index.isin(adata.obs.index)]) == list(pred_out.index)

In [ ]:
adata.obs['predicted'] = pred_out['predicted']

In [ ]:
key_add = 'leiden_res5'
key_add2 = key_add + '_sub_clust_res_1'
adata.obs[key_add2] = adata.obs[key_add].astype('category')
for cluster in adata.obs[key_add2].unique():
    if len(adata.obs[adata.obs[key_add2].isin([cluster])]) > 10: # will error with 1 cell
        sc.tl.leiden(adata, resolution= 1, key_added=key_add2, restrict_to=[key_add2, [cluster]])

In [ ]:
pred_out['leiden_res5_sub_clust_res_1'] = adata.obs['leiden_res5_sub_clust_res_1']

In [ ]:
cluster_prediction = "clus_prediction"
clusters_reassign = "leiden_res5_sub_clust_res_1"
lr_predicted_col = 'predicted'
reassign_classes = list(pred_out['leiden_res5_sub_clust_res_1'].unique())

lm = 1 # lambda value
pred_out[cluster_prediction] = pred_out[clusters_reassign]
for z in pred_out[clusters_reassign][pred_out[clusters_reassign].isin(reassign_classes)].unique():
    df = pred_out
    df = df[(df[clusters_reassign].isin([z]))]
    df_count = pd.DataFrame(df[lr_predicted_col].value_counts())
    # Look for classificationds > 68CI
    if len(df_count) > 1:
        df_count_temp = df_count[df_count[lr_predicted_col]>int(int(df_count.mean()) + (df_count.std()*lm))]
        if len(df_count_temp >= 1):
            df_count = df_count_temp
    #print(df_count)     
    freq_arranged = df_count.index
    cat = freq_arranged[0]
    #df.loc[:,cluster_prediction] = cat
#Make the cluster assignment first
    cat = cat.replace(data1,'') + '_clus_prediction'
    pred_out[cluster_prediction] = pred_out[cluster_prediction].astype(str)
    pred_out.loc[pred_out[clusters_reassign] == z, [cluster_prediction]] = cat
    #print(cat)
# Create assignments for any classification >68CI
    for cats in freq_arranged:
        #print(cats)
        cats_assignment = cats.replace(data1,'') + '_clus_prediction'
        pred_out.loc[(pred_out[clusters_reassign] == z) & (pred_out[lr_predicted_col] == cats),[cluster_prediction]] = cats_assignment
min_counts = pd.DataFrame((pred_out[cluster_prediction].value_counts()))
reassign = list(min_counts.index[min_counts[cluster_prediction]<=2])
pred_out[cluster_prediction] = pred_out[cluster_prediction].str.replace(str(''.join(reassign)),str(''.join(pred_out.loc[pred_out[clusters_reassign].isin(list(pred_out.loc[(pred_out[cluster_prediction].isin(reassign)),clusters_reassign])),lr_predicted_col].value_counts().head(1).index.values)))#.replace(data1,'') + '_clus_prediction')

In [ ]:
adata.obs['clus_prediction'] = pred_out[cluster_prediction]
adata.obs['clus_prediction'].value_counts(dropna=False)

In [ ]:
adata.obs['predicted'].value_counts(dropna=False)

In [ ]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [ ]:
df_check = adata.obs[adata.obs['predicted'].isin(['Macrophage_training','Microglia_training'])]
df_check.groupby(['leiden_res5','clus_prediction']).apply(len)

In [ ]:
df_check['predicted'].value_counts(dropna=False)

In [ ]:
df_check2 = df_check[df_check['clus_prediction'].isin(['Macrophage_clus_prediction','Microglia_clus_prediction'])]

In [ ]:
df_check2.groupby(['leiden_res5_sub_clust_res_1','clus_prediction']).apply(len)

In [ ]:
df_check3 = df_check[df_check['clus_prediction'].isin(['Microglia_clus_prediction'])]

In [ ]:
df_check3.groupby(['leiden_res5_sub_clust_res_1','clus_prediction']).apply(len)

In [ ]:
sc.pl.dotplot(adata, groupby='predicted', var_names=genes_to_plot)

In [ ]:
sc.pl.dotplot(adata, groupby='clus_prediction', var_names=genes_to_plot)

In [ ]:
adata.obs['Microglia_check_res_20_clus_prediction'] = metadata['Microglia_check_res_20_clus_prediction']

In [ ]:
sc.pl.dotplot(adata, groupby='Microglia_check_res_20_clus_prediction', var_names=genes_to_plot)

# new clus prediction looks better than model ran on broad

In [ ]:
adata.obs['predicted'].value_counts()

In [ ]:
adata.obs['clus_prediction'].value_counts()

In [ ]:
adata.obs['Microglia_check_res_20_clus_prediction'].value_counts()

# probably got so few microglia back due to number of other additional celltypes affecting model probabilities as too complex

In [ ]:
x = 'predicted'
y =  "clus_prediction"

y_attr = adata.obs[y]
x_attr = adata.obs[x]
crs = pd.crosstab(x_attr, y_attr)
crs_tbl = crs
for col in crs_tbl :
    crs_tbl[col] = crs_tbl[col].div(crs_tbl[col].sum(axis=0)).multiply(100).round(2)
    
# Sort df columns by rows
crs_tbl = crs_tbl.sort_values(by =list(crs_tbl.index), axis=1,ascending=False)

# Plot_df_heatmap(crs_tbl, cmap='coolwarm', rotation=90, vmin=20, vmax=70)
pal = sns.diverging_palette(240, 10, n=10)
plt.figure(figsize=(20,15))
sns.set(font_scale=0.8)
g = sns.heatmap(crs_tbl[['Macrophage_clus_prediction','Microglia_clus_prediction']], cmap=pal, vmin=0, vmax=100, linewidths=1, center=50, square=True, cbar_kws={"shrink": 0.5}, annot=False)
plt.xlabel("Original labels")
plt.ylabel("Predicted labels")
plt.savefig(save_path + "/brain_LR_predictions_myeloid_clus_predict_labels_20220405.pdf")
crs_tbl.to_csv(save_path + "/brani_crs_tbl_myeloid_clus_predict_labels_20220405.csv")

# Cut for figure

In [ ]:
ordering = ['MOP_training', 'Promonocyte_training','Monocyte_training','Pre_Macrophage_training','Macrophage_training','Microglia_training']

In [ ]:
crs_tbl2 =crs_tbl.copy()
crs_tbl2 = crs_tbl2[crs_tbl2.index.isin(ordering)]
crs_tbl2 = crs_tbl2[['Macrophage_clus_prediction','Microglia_clus_prediction']]
crs_tbl2

In [ ]:
crs_tbl2 = crs_tbl2.T

In [ ]:
crs_tbl2

In [ ]:
mono_row = {'Macrophage_clus_prediction':0, 'Microglia_clus_prediction':0}

In [ ]:
crs_tbl2.loc['Monocyte_training'] = mono_row

In [ ]:
crs_tbl2

In [ ]:
crs_tbl2 = crs_tbl2.T
crs_tbl2 = crs_tbl2[ordering]
crs_tbl2 = crs_tbl2.T
crs_tbl2

In [ ]:
# Plot_df_heatmap(crs_tbl, cmap='coolwarm', rotation=90, vmin=20, vmax=70)
pal = sns.diverging_palette(240, 10, n=10)
plt.figure(figsize=(20,15))
sns.set(font_scale=0.8)
g = sns.heatmap(crs_tbl2, cmap=pal, vmin=0, vmax=100, linewidths=1, center=50, square=True, cbar_kws={"shrink": 0.5}, annot=False)
plt.xlabel("Original labels")
plt.ylabel("Predicted labels")
plt.savefig(save_path + "/brain_skin_probability_heatmap_20220405.pdf")
crs_tbl.to_csv(save_path + "/crs_tbl_YS_brain_probability_20220405.csv")

In [ ]:
adata.obs.to_csv('brain_microglia_clean_metadata_20220405.csv')

In [ ]:
pred_out.to_csv('final_pred_out_brain_20220405.csv')

In [ ]:
adata.write('brain_with_clean_microglia_post_LR_20220405.h5ad')